Будет разработана специальная функция для abc-анализа, а именно классификация товаров по степени их важности. Данная функция будет принимать на вход в качестве аргументов DataFrame и имя столбца, в котором находятся анализируемые объекты. DataFrame должен быть заранее обработан, т.е. содержать только столбец с анализируемыми объектами и столбцы с необходимыми метриками.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('sales_products_abc_obr.csv')

In [37]:
def abc(dataframe, index):
    # получаем список названий столбцов, удаляем из списка столбец с анализируемыми объектами
    agg_columns = list(dataframe.columns)
    agg_columns.remove(index)
    # запускаем группировку и агрегацию по стобцам из списка
    df_grupped = dataframe.groupby(index, as_index=False)[agg_columns].sum()
    # запускам цискл по агрегируемым столбцам для рассчета доли товара
    for i in range(len(agg_columns)):
        df_grupped[agg_columns[i]] = df_grupped.sort_values(agg_columns[i], ascending=False)\
        [agg_columns[i]].cumsum() / sum(df_grupped[agg_columns[i]])
    # запускам цискл по агрегируемым столбцам для проставления группы продукта в зависимости от общего вклада
    for i in range(len(agg_columns)):
        df_grupped[agg_columns[i]] = np.where(df_grupped[agg_columns[i]] < 0.7, 'A', np.where(df_grupped[agg_columns[i]] < 0.95, 'B', 'C'))
    #делаем сортировку предитоговой таблицы по первому столбцу
    df_grupped_sort = df_grupped.sort_values(agg_columns[0])
    # запускам функцию для заливки ячеек с группами товаров и обеспечения наглядности и ранжирование по первому столбцу
    df_grupped_sort =  df_grupped_sort.style.map\
    (lambda x: 'background-color: lightgreen' if x == 'A' else 'background-color: pink' if x == 'C' \
    else 'background-color: yellow' ,subset=agg_columns)
    return df_grupped_sort

In [38]:
abc(df, 'product')

,product,quantity,revenue,profit
14,Mouse,A,A,A
21,SSD,A,A,A
17,Processor,A,A,B
22,Smart Light,A,A,A
23,Smartphone,A,A,A
28,USB Drive,A,A,A
11,Microphone,A,B,B
19,RAM,A,A,A
26,TV,A,A,A
10,Laptop,A,A,A


Подводим итоги по полученной таблице
Товары A по всем метрикам: Mouse, SSD, Smart Light, USB Drive, Smartphone, Laptop, TV, Graphics Card, Game Console, Tablet, External HDD, E-book Reader, Drone, Charger, Headphones, RAM Эти товары — золотой фонд. Их стоит: держать в фокусе маркетинга / не допускать дефицита /возможно, расширять ассортимент в этих категориях.

Интересные случаи:
Speaker — B по количеству, но A по выручке и прибыли → Может продаваться реже, но по высокой цене и марже. Router — A по продажам и прибыли, но B по выручке → Часто продаётся, но возможно, по невысокой цене. Processor — A по продажам и выручке, но B по прибыли → Хороший оборот, но маржа низкая — стоит пересмотреть цену или закупку. Microphone — A по продажам, но B по выручке и прибыли → Объем есть, но прибыль низкая — может стоит сделать апсейл.

Класс C — слабые звенья:
Monitor, Keyboard, Motherboard, Fitness Tracker, Webcam → Продаются нечасто, приносят мало денег и прибыли.

Рекомендации:
проанализировать: это запасной ассортимент или залежавшийся? подумать о распродаже или выводе / попробовать связать с другими товарами (бандлы, апсейл).

Вывод  
Стоит сфокусировать усилия на товарах из категории A/A/A. Проверять несоответствия: если товар хорошо продаётся, но не приносит прибыль — пересмотреть ценообразование. Товары из категории C — кандидаты на оптимизацию ассортимента или вывод из ассортимента.